In [1]:
import numpy as np
import cv2
import tensorflow as tf
import h5py
import winsound

face_cascade = cv2.CascadeClassifier('C:\\Users\\vvikk\\anaconda3\\Lib\\site-packages\\cv2\\data\\haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('C:\\Users\\vvikk\\anaconda3\\Lib\\site-packages\\cv2\\data\\haarcascade_eye.xml')

In [2]:
def face_detect(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    if len(faces)==0:
        eyes = 0
        faces = 0
        pred = 1
        return frame,eyes,faces,pred
    for (x,y,w,h) in faces:                                                       
        img = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)          
        roi_gray = gray[y:y+h, x:x+w] 
        roi_color = img[y:y+h, x:x+w]                                   
        eyes = eye_cascade.detectMultiScale(roi_gray,1.1, 5)
        if len(eyes)==0:
            eyes = 0
            pred = 1
            return frame,eyes,faces,pred
        for (ex,ey,ew,eh) in eyes: 
            detected = cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            gray = cv2.cvtColor(roi_color, cv2.COLOR_BGR2GRAY)
            pred_img = cv2.resize(gray[ey:ey+eh,ex:ex+ew],(50,50))
            trail = np.expand_dims(pred_img,axis=-1)
            pred = model.predict(np.array([trail]))
            pred+=tmp
            
    return frame,eyes,faces,pred

In [3]:
model = tf.keras.models.load_model('close_open2.h5')
cap = cv2.VideoCapture(0)
detect = 0

while(True):
    tmp = 0
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    frame,eyes,faces,pred = face_detect(frame)
    
    try:
        for fx,fy,fw,fh in faces:
            if pred <0.5:
                cv2.putText(frame, 'eyes_close', (fx, fy-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
                detect += 1
                if detect == 10:
                    winsound.PlaySound("sleep.wav", winsound.SND_ASYNC|winsound.SND_ALIAS)
            else:
                detect = 0
    except:
        pass

    # Display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()